In [ ]:
import jax.numpy as jnp
import jax.random as jrand

import genjax
from genjax import gen, pjax_to_jax


@gen
def model(carry, xs):
    x = genjax.normal(carry, 1.0) @ "x"
    return x, xs


pjax_to_jax(jrand.key(1), model.scan(n=10).simulate)((0.1, jnp.ones(10)))